# 生成式对抗网络(GAN)

In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from datetime import timedelta
from numpy import ndarray
from typing import Union, List, Dict
from sklearn.preprocessing import MinMaxScaler
from ultralytics import YOLO
import cv2
from PIL import Image
import os
import json
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Subset
import torch.nn.functional as F
from tqdm import tqdm  # 打印进度条
import math
import matplotlib.pyplot as plt
from torchvision import datasets, transforms, models
import torchvision.utils as vutils
import seaborn as sns
from typing import List
from pandas.tseries import offsets
from pandas.tseries.frequencies import to_offset
from sklearn.metrics import r2_score, mean_squared_error
import joblib
import warnings

warnings.filterwarnings("ignore")
plt.rcParams['font.sans-serif'] = ['SimHei']  # 显示中文标签
plt.rcParams['axes.unicode_minus'] = False
%matplotlib inline

## 基本概念

生成式对抗网络迫使生成图像与真实图像在统计上几乎无法区别，从而生成相当逼真的合成图像。GAN由一个生成器网络(generator)和一个判别式网络(discriminator)组成。判别器的训练目的是能够区分生成器的输出与来自训练集的真实图像，生成器的训练目的是欺骗判别器。生成器从未直接见过训练集中的图像，它所知道的关于数据的信息都来自于判别器。

生成器生成假数据，然后将生成的假数据和真数据都输入判别器，判别器要判断出哪些是真的哪些是假的。判别器第一次判别出来的肯定有很大的误差，然后我们根据误差来优化判别器。现在判别器水平提高了，生成器生成的数据很难再骗过判别器了，所以我们得反过来优化生成器，之后生成器水平提高了，然后反过来继续训练判别器，判别器水平又提高了，再反过来训练生成器，就这样循环往复，直到达到纳什均衡。

**生成网络的损失函数：**
$$L_G=H(1,D(G(z)))$$
上式中，$G$ 代表生成网络，$D $代表判别网络，$H$ 代表交叉熵，$z$ 是输入随机数据。$D(G(z))$是对生成数据的判断概率，1代表数据绝对真实，0代表数据绝对虚假。$H(1,D(G(z)))$代表判断结果与1的距离。显然生成网络想取得良好的效果，那就要做到，让判别器将生成数据判别为真数据（即$D(G(z))$与1的距离越小越好）。

**判别网络的损失函数：**
$$L_D=H(1,D(x))+H(0,D(G(z)))$$
上式中，$x$是真实数据，这里要注意的是，$H(1,D(x))$代表真实数据与1的距离，$H(0,D(G(z)))$代表生成数据与0的距离。显然，识别网络要想取得良好的效果，那么就要做到，在它眼里，真实数据就是真实数据，生成数据就是虚假数据（即真实数据与1的距离小，生成数据与0的距离小）。
 

**理想的损失趋势**
- 生成器的损失 (Generator Loss)：

    - 初期较高，随着训练的进行逐渐下降。
    - 稳定后维持在一个较低水平，但不是接近 0，因为生成器需要不断与判别器竞争。
- 判别器的损失 (Discriminator Loss)：

    - 初期较低，表示判别器能够轻松区分真实样本和生成样本。
    - 随着生成器的改进，判别器的损失逐渐上升，趋于约 0.5（随机猜测的水平）。
  
两者在一个理想的平衡点上达到动态稳定：生成器和判别器互相逼近彼此的最优性能。

**优化原理**：生成网络和判别网络有了损失函数，就可以基于各自的损失函数，利用误差反向传播（Backpropagation）(BP)反向传播算法和最优化方法(如梯度下降法)来实现参数的调整），不断提高生成网络和判别网络的性能（最终生成网络和判别网络的成熟状态就是学习到了合理的映射函数）。

### DCGAN

## 数据准备

In [2]:
# 加载图像并生成批次数据
def generator(data_path, batch_size):
    """
    读取图像，并生成批次数据

    参数说明
    ----------
    data_path : {str}
        图像文件夹地址
    batch_size : {int} 
        输入数据的批次大小，正整数

    返回值
    -------
    data_loader : {torch.utils.data.dataloader.DataLoader}
        数据加载器，[批次，目标，特征时间编码，目标时间编码]
    """
    # 定义图像变换操作
    transform = transforms.Compose([
        transforms.Resize((224, 224)),         # 调整图像大小
        transforms.ToTensor(),                  # 转换为张量
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) # 标准化
    ])
    
    # dataset = datasets.CIFAR10(root=data_path, download=True, transform=transform)
    # # 筛选标签为6的索引
    # indices = [i for i, (_, label) in enumerate(dataset) if label == 6]
    # # 创建只包含标签为6数据的子集
    # dataset = Subset(dataset, indices)
    
    dataset = datasets.ImageFolder(root=data_path, transform=transform)
    print(f"图像个数：{len(dataset)}, 尺寸：{dataset[0][0].shape}")
    
    # 数据加载器
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # 查看一个批次数据
    images, labels = next(iter(dataloader))
    print(f"图像批次大小: {images.shape}")
    print(f"标签批次大小: {labels.shape}")
    print(f"图像批次个数: {len(dataloader)}")

    return dataloader

In [3]:
# 训练集
params1 = {
    "data_path": "../data/cats_and_dogs_small/train/",
    "batch_size": 16,
}
print("训练集：")
data_loader = generator(**params1)

训练集：
图像个数：2000, 尺寸：torch.Size([3, 224, 224])
图像批次大小: torch.Size([16, 3, 224, 224])
标签批次大小: torch.Size([16])
图像批次个数: 125


## 模型定义

### 自定义模型

In [20]:
# 定义生成器网络
class Generator(nn.Module):
    def __init__(self, latent_dim, channels):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            # 1. 全连接层：将潜在向量 (latent_dim) 投影并展平为形状 (128, 8, 8)
            nn.Linear(latent_dim, 512 * 8 * 8),
            nn.LeakyReLU(0.3, inplace=True),
            nn.Unflatten(1, (512, 8, 8)),  # 转换为形状 (batch, 128, 8, 8)

            # 2. 转置卷积层：第一次上采样，(128, 8, 8) -> (128, 16, 16)
            nn.ConvTranspose2d(512, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.3, inplace=True),

            # 3. 转置卷积层：第二次上采样，(128, 16, 16) -> (64, 32, 32)
            nn.ConvTranspose2d(512, 224, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(224),
            nn.LeakyReLU(0.3, inplace=True),

            # 4. 卷积层：输出通道调整为图像通道数 (64, 32, 32) -> (3, 32, 32)
            nn.Conv2d(224, channels, kernel_size=3, stride=1, padding=1),
            nn.Tanh()  # 将输出范围限制在 [-1, 1]
        )

    def forward(self, z):
        return self.model(z)

# class Generator(nn.Module):
#     def __init__(self, latent_dim, channels):
#         super(Generator, self).__init__()
#         self.model = nn.Sequential(
#             # 1. 全连接层，生成16*16*128的特征图，输出维度：(batch_size, 128 * 16 * 16)
#             nn.Linear(latent_dim, 128 * 16 * 16),  
#             nn.LeakyReLU(0.2, inplace=True),
#             nn.Unflatten(1, (128, 16, 16)),  # 重塑为特征图，输出维度：(batch_size, 128, 16, 16)
            
#             # 2. 卷积操作，输出维度：(batch_size, 256, 16, 16)
#             nn.Conv2d(128, 256, kernel_size=5, padding=2),  
#             nn.LeakyReLU(0.2, inplace=True),

#             # 3. 上采样为32x32，输出维度：(batch_size, 256, 32, 32)
#             nn.ConvTranspose2d(256, 256, kernel_size=4, stride=2, padding=1),  
#             nn.LeakyReLU(0.2, inplace=True),

#             # 4. 卷积操作，输出维度：(batch_size, 256, 32, 32)
#             nn.Conv2d(256, 256, kernel_size=5, padding=2),  
#             nn.LeakyReLU(0.2, inplace=True),

#             # 5. 再次卷积操作，输出维度：(batch_size, 256, 32, 32)
#             nn.Conv2d(256, 256, kernel_size=5, padding=2),  
#             nn.LeakyReLU(0.2, inplace=True),

#             # 6. 输出图像，padding=3保持输出尺寸不变，输出维度：(batch_size, channels, 32, 32)
#             nn.Conv2d(256, channels, kernel_size=7, padding=3),  
#             nn.Tanh()  # 输出激活函数
#         )

#     def forward(self, x):
#         return self.model(x)

# 定义判别器网络
class Discriminator(nn.Module):
    def __init__(self, channels):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            # 1. 卷积层：初始通道从图像通道数增加到 128，(3, 32, 32) -> (128, 32, 32)
            nn.Conv2d(channels, 128, 3, padding=1),
            nn.LeakyReLU(0.2, inplace=True),

            # 2. 卷积层：下采样，(128, 32, 32) -> (128, 16, 16)
            nn.Conv2d(128, 128, 4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),

            # 3. 卷积层：下采样，(128, 16, 16) -> (128, 8, 8)
            nn.Conv2d(128, 128, 4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),

            # 4. 卷积层：下采样，(128, 8, 8) -> (128, 4, 4)
            nn.Conv2d(128, 128, 4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),

            # 5. 展平层：将 (128, 4, 4) 展平为向量 (128 * 4 * 4)
            nn.Flatten(),
            nn.Dropout(0.4),  # 添加 Dropout 防止过拟合

            # 6. 全连接层：映射到单个输出值 (即概率)
            nn.Linear(128 * 31 * 31, 1),
            nn.Sigmoid()  # 使用 Sigmoid 输出范围在 [0, 1]
        )

    def forward(self, img):
        return self.model(img)

In [26]:
class Generator(nn.Module):
    def __init__(self, latent_dim, channels):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            # 全连接层：将潜在向量映射为形状 (512, 14, 14)
            nn.Linear(latent_dim, 512 * 14 * 14),
            nn.LeakyReLU(0.3, inplace=True),
            nn.Unflatten(1, (512, 14, 14)),  # 转换为形状 (batch, 512, 14, 14)

            # 转置卷积层：第一次上采样 (512, 14, 14) -> (256, 28, 28)
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.3, inplace=True),

            # 第二次上采样 (256, 28, 28) -> (128, 56, 56)
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.3, inplace=True),

            # 第三次上采样 (128, 56, 56) -> (64, 112, 112)
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.3, inplace=True),

            # 第四次上采样 (64, 112, 112) -> (channels, 224, 224)
            nn.ConvTranspose2d(64, channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh()  # 输出范围限制在 [-1, 1]
        )

    def forward(self, z):
        return self.model(z)

class Discriminator(nn.Module):
    def __init__(self, channels):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            # 卷积层：初始通道从图像通道数增加到 64，(3, 224, 224) -> (64, 112, 112)
            nn.Conv2d(channels, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),

            # 卷积层：下采样，(64, 112, 112) -> (128, 56, 56)
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            # 卷积层：下采样，(128, 56, 56) -> (256, 28, 28)
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            # 卷积层：下采样，(256, 28, 28) -> (512, 14, 14)
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            # 卷积层：下采样，(512, 14, 14) -> (1024, 7, 7)
            nn.Conv2d(512, 1024, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.2, inplace=True),

            # 展平层：将 (1024, 7, 7) 展平为向量 (1024 * 7 * 7)
            nn.Flatten(),
            nn.Dropout(0.4),

            # 全连接层：映射到单个输出值 (即概率)
            nn.Linear(1024 * 7 * 7, 1),
            nn.Sigmoid()  # 输出范围在 [0, 1]
        )

    def forward(self, img):
        return self.model(img)


### 预训练模型

In [13]:
class Discriminator(nn.Module):
    def __init__(self, channels):
        super(Discriminator, self).__init__()
        vgg = models.vgg16(pretrained=True)

        # 如果输入通道数不为3，则添加自定义的卷积层
        if channels != 3:
            self.input_layer = nn.Conv2d(channels, 3, kernel_size=3, stride=1, padding=1)
        else:
            self.input_layer = nn.Identity()  # 不改变输入

        self.feature_extractor = nn.Sequential(*list(vgg.features[:10]))
        self.fc = nn.Sequential(
            nn.Linear(512 * 14 * 14, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        img = self.input_layer(img)  # 调整输入通道数
        with torch.no_grad():  # 冻结预训练模型的参数
            features = self.feature_extractor(img)
        features = features.view(features.size(0), -1)
        print(features.shape)
        validity = self.fc(features)
        return validity

## 模型训练

In [27]:
def train(train_args, generator_args, discriminator_args):
    # 参数配置
    generator_name = train_args['generator_name']  # 生成器模型名称
    discriminator_name = train_args['discriminator_name']  # 判别器模型名称
    data_loader = train_args['data_loader']  # 训练集
    n_epochs = train_args['n_epochs']  # 训练次数
    learning_rate = train_args['learning_rate']  # 学习率
    lradj = train_args['lradj'] # 学习率函数
    model_path = train_args['model_path']  # 模型保存路径
    image_path = train_args['image_path']  # 生成图像路径
    verbose = train_args['verbose']  # 打印训练过程
    plots = train_args['plots']  # 绘制损失图
    device = train_args['device']  # 可选'cuda'和'cpu'
    clip_value = train_args['clip_value'] # 裁剪值
    loss = train_args['loss'] # 损失函数
    latent_dim = generator_args['latent_dim'] # 潜在向量维度，图像尺寸
    channels = generator_args['channels'] # 颜色通道数

    # 检查可用device
    device = torch.device(device)

    # 调整学习率
    def adjust_learning_rate(optimizer, epoch, lradj, learning_rate, train_epochs):
        # lr = learning_rate * (0.2 ** (epoch // 2))
        if lradj == 'type1':
            lr_adjust = {epoch: learning_rate * (0.5 ** ((epoch - 1) // 1))}
        elif lradj == 'type2':
            lr_adjust = {
                2: 5e-5, 4: 1e-5, 6: 5e-6, 8: 1e-6,
                10: 5e-7, 15: 1e-7, 20: 5e-8
            }
        elif lradj == "cosine":
            lr_adjust = {epoch: learning_rate /2 * (1 + math.cos(epoch / train_epochs * math.pi))}
        if epoch in lr_adjust.keys():
            lr = lr_adjust[epoch]
            # 参数组(param_groups)是用来指定不同的参数组以便对它们进行不同的优化设置，比如'lr'
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
            print('Updating learning rate to {}'.format(lr))

    # 设置保存模型路径
    if not os.path.exists(model_path):
        os.makedirs(model_path)

    # 定义模型和损失函数
    generator = generator_name(**generator_args).to(device)
    discriminator = discriminator_name(**discriminator_args).to(device)

    optimizer_G = optim.Adam(generator.parameters(), lr=learning_rate, betas=(0.5, 0.999))
    optimizer_D = optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(0.5, 0.999))
    criterion = loss

    # 损失函数值
    G_losses, D_losses = [], []

    for epoch in tqdm(range(n_epochs)):
        # generator.train()
        # discriminator.train()
        total_G_loss = 0
        total_D_loss = 0
        for i, (imgs, _) in enumerate(data_loader):
            # 将输入数据移至device
            imgs = imgs.to(device)

            # 创建真实标签和假标签
            real_labels = torch.ones(imgs.size(0), 1).to(device)  # 真实图像标签为 1
            fake_labels = torch.zeros(imgs.size(0), 1).to(device) # 生成图像标签为 0

            # 训练判别器
            z = torch.randn(imgs.size(0), latent_dim).to(device)  # 从标准正态分布中采样潜在向量
            fake_imgs = generator(z)  # 使用生成器生成假图像
    
            real_loss = criterion(discriminator(imgs), real_labels)  # 判别器对真实图像的损失
            fake_loss = criterion(discriminator(fake_imgs.detach()), fake_labels)  # 判别器对假图像的损失
            loss_D = real_loss + fake_loss  # 判别器总损失
    
            optimizer_D.zero_grad()  # 清空梯度
            loss_D.backward()  # 反向传播
            torch.nn.utils.clip_grad_norm_(discriminator.parameters(), clip_value) # 判别器梯度裁剪
            optimizer_D.step()  # 更新判别器参数
    
            # 训练生成器
            loss_G = criterion(discriminator(fake_imgs), real_labels)  # 生成器希望生成的图像被判别为真实
    
            optimizer_G.zero_grad()  # 清空梯度
            loss_G.backward()  # 反向传播
            torch.nn.utils.clip_grad_norm_(generator.parameters(), clip_value) # 生成器梯度裁剪
            optimizer_G.step()  # 更新生成器参数

            # 计算每个batch的loss和
            total_G_loss += loss_G.item()
            total_D_loss += loss_D.item()

        # 计算每个epoch的损失平均
        avg_G_loss = total_G_loss / len(data_loader)
        avg_D_loss = total_D_loss / len(data_loader)

        # 记录所有epoch的loss
        G_losses.append(avg_G_loss)
        D_losses.append(avg_D_loss)

        # 打印训练过程
        print(
            f'Epoch [{epoch}/{n_epochs}], Generator Loss: {avg_G_loss:.4f}, Discriminator Loss: {avg_D_loss:.4f}')

        # 每个 epoch 保存生成的图片
        fake_imgs = fake_imgs.view(fake_imgs.size(0), channels, latent_dim, latent_dim)
        vutils.save_image(fake_imgs[:25], f"{image_path}/output_epoch_{epoch+1}.png", nrow=5, normalize=True)

        # 调整学习率
        adjust_learning_rate(optimizer_G, epoch+1, lradj, learning_rate, n_epochs)
        adjust_learning_rate(optimizer_D, epoch+1, lradj, learning_rate, n_epochs)

    # 绘制损失函数图
    def plot_loss(G_losses, D_losses):
        plt.figure(figsize=(10, 5))
        plt.style.use('seaborn-v0_8-paper')  # 绘制背景色
        plt.grid(axis='y', linewidth=0.35)  # 绘制网格
        plt.plot(G_losses, linestyle='-', color='#11b3b6')
        plt.plot(D_losses, linestyle='-', color='#f14643')
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.title("Training and Validation Progress")
        plt.legend(["Generator", "Discriminator"])
        plt.show()
    if plots:
        plot_loss(G_losses, D_losses)

    return generator

In [28]:
# 构造参数字典
params2 = {
    "train_args": {
        "generator_name": Generator,
        "discriminator_name": Discriminator,
        "data_loader": data_loader,
        "n_epochs": 100,
        "patience": 50,
        "learning_rate": 0.0001,
        "lradj": 'cosine',
        "model_path": "../outputs/best_models/DCGAN",
        "image_path": "../outputs/images",
        "device": 'cuda',
        "loss": nn.BCELoss(),
        "verbose": True,
        "plots": True,
        "clip_value": 1.0,
    },
    "generator_args": {
        'latent_dim': 224,
        'channels': 3,
    },
    "discriminator_args": {
        'channels': 3,
    },
}
generator = train(**params2)

  0%|                                                                                          | 0/100 [16:17<?, ?it/s]


KeyboardInterrupt: 

## 图像生成

In [ ]:
def train(train_args, generator_args, discriminator_args):
    # 参数配置
    generator_name = train_args['generator_name']  # 生成器模型名称
    discriminator_name = train_args['discriminator_name']  # 判别器模型名称
    data_loader = train_args['data_loader']  # 训练集
    n_epochs = train_args['n_epochs']  # 训练次数
    learning_rate = train_args['learning_rate']  # 学习率
    lradj = train_args['lradj'] # 学习率函数
    model_path = train_args['model_path']  # 模型保存路径
    image_path = train_args['image_path']  # 生成图像路径
    verbose = train_args['verbose']  # 打印训练过程
    plots = train_args['plots']  # 绘制损失图
    device = train_args['device']  # 可选'cuda'和'cpu'
    clip_value = train_args['clip_value'] # 裁剪值
    loss = train_args['loss'] # 损失函数
    latent_dim = generator_args['latent_dim'] # 潜在向量维度，图像尺寸
    channels = generator_args['channels'] # 颜色通道数

    # 检查可用device
    device = torch.device(device)

    # 调整学习率
    def adjust_learning_rate(optimizer, epoch, lradj, learning_rate, train_epochs):
        # lr = learning_rate * (0.2 ** (epoch // 2))
        if lradj == 'type1':
            lr_adjust = {epoch: learning_rate * (0.5 ** ((epoch - 1) // 1))}
        elif lradj == 'type2':
            lr_adjust = {
                2: 5e-5, 4: 1e-5, 6: 5e-6, 8: 1e-6,
                10: 5e-7, 15: 1e-7, 20: 5e-8
            }
        elif lradj == "cosine":
            lr_adjust = {epoch: learning_rate /2 * (1 + math.cos(epoch / train_epochs * math.pi))}
        if epoch in lr_adjust.keys():
            lr = lr_adjust[epoch]
            # 参数组(param_groups)是用来指定不同的参数组以便对它们进行不同的优化设置，比如'lr'
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
            print('Updating learning rate to {}'.format(lr))

    # 设置保存模型路径
    if not os.path.exists(model_path):
        os.makedirs(model_path)

    # 定义模型和损失函数
    generator = generator_name(**generator_args).to(device)
    discriminator = discriminator_name(**discriminator_args).to(device)

    optimizer_G = optim.Adam(generator.parameters(), lr=learning_rate, betas=(0.5, 0.999))
    optimizer_D = optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(0.5, 0.999))
    criterion = loss

    # 损失函数值
    G_losses, D_losses = [], []

    for epoch in tqdm(range(n_epochs)):
        # generator.train()
        # discriminator.train()
        total_G_loss = 0
        total_D_loss = 0
        for i, (imgs, _) in enumerate(data_loader):
            # 将输入数据移至device
            imgs = imgs.to(device)

            # 创建真实标签和假标签
            real_labels = torch.ones(imgs.size(0), 1).to(device)  # 真实图像标签为 1
            fake_labels = torch.zeros(imgs.size(0), 1).to(device) # 生成图像标签为 0

            # 训练判别器
            z = torch.randn(imgs.size(0), latent_dim).to(device)  # 从标准正态分布中采样潜在向量
            fake_imgs = generator(z)  # 使用生成器生成假图像
    
            real_loss = criterion(discriminator(imgs), real_labels)  # 判别器对真实图像的损失
            fake_loss = criterion(discriminator(fake_imgs.detach()), fake_labels)  # 判别器对假图像的损失
            loss_D = real_loss + fake_loss  # 判别器总损失
    
            optimizer_D.zero_grad()  # 清空梯度
            loss_D.backward()  # 反向传播
            torch.nn.utils.clip_grad_norm_(discriminator.parameters(), clip_value) # 判别器梯度裁剪
            optimizer_D.step()  # 更新判别器参数
    
            # 训练生成器
            loss_G = criterion(discriminator(fake_imgs), real_labels)  # 生成器希望生成的图像被判别为真实
    
            optimizer_G.zero_grad()  # 清空梯度
            loss_G.backward()  # 反向传播
            torch.nn.utils.clip_grad_norm_(generator.parameters(), clip_value) # 生成器梯度裁剪
            optimizer_G.step()  # 更新生成器参数

            # 计算每个batch的loss和
            total_G_loss += loss_G.item()
            total_D_loss += loss_D.item()

        # 计算每个epoch的损失平均
        avg_G_loss = total_G_loss / len(data_loader)
        avg_D_loss = total_D_loss / len(data_loader)

        # 记录所有epoch的loss
        G_losses.append(avg_G_loss)
        D_losses.append(avg_D_loss)

        # 打印训练过程
        print(
            f'Epoch [{epoch}/{n_epochs}], Generator Loss: {avg_G_loss:.4f}, Discriminator Loss: {avg_D_loss:.4f}')

        # 每个 epoch 保存生成的图片
        fake_imgs = fake_imgs.view(fake_imgs.size(0), channels, latent_dim, latent_dim)
        vutils.save_image(fake_imgs[:25], f"{image_path}/output_epoch_{epoch+1}.png", nrow=5, normalize=True)

        # 调整学习率
        adjust_learning_rate(optimizer_G, epoch+1, lradj, learning_rate, n_epochs)
        adjust_learning_rate(optimizer_D, epoch+1, lradj, learning_rate, n_epochs)

    # 绘制损失函数图
    def plot_loss(G_losses, D_losses):
        plt.figure(figsize=(10, 5))
        plt.style.use('seaborn-v0_8-paper')  # 绘制背景色
        plt.grid(axis='y', linewidth=0.35)  # 绘制网格
        plt.plot(G_losses, linestyle='-', color='#11b3b6')
        plt.plot(D_losses, linestyle='-', color='#f14643')
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.title("Training and Validation Progress")
        plt.legend(["Generator", "Discriminator"])
        plt.show()
    if plots:
        plot_loss(G_losses, D_losses)

    return generator

In [ ]:
def train(train_args, generator_args, discriminator_args):
    # 参数配置
    generator_name = train_args['generator_name']
    discriminator_name = train_args['discriminator_name']
    data_loader = train_args['data_loader']
    n_epochs = train_args['n_epochs']
    learning_rate = train_args['learning_rate']
    lradj = train_args['lradj']
    model_path = train_args['model_path']
    image_path = train_args['image_path']
    verbose = train_args['verbose']
    plots = train_args['plots']
    device = train_args['device']
    clip_value = train_args['clip_value']
    loss = train_args['loss']
    latent_dim = generator_args['latent_dim']
    channels = generator_args['channels']
    noise_level = train_args.get('noise_level', 0.1)  # 默认噪声强度

    # 检查可用device
    device = torch.device(device)

    # 添加噪声
    def add_noise(images, noise_level=0.1):
        """
        对图像添加随机噪声
        :param images: 输入图像张量
        :param noise_level: 噪声强度（默认0.1）
        :return: 添加噪声后的图像张量
        """
        noise = torch.randn_like(images) * noise_level
        noisy_images = images + noise
        return noisy_images.clamp(0.0, 1.0)  # 保证像素值在[0, 1]范围内

    # 调整学习率
    def adjust_learning_rate(optimizer, epoch, lradj, learning_rate, train_epochs):
        if lradj == 'type1':
            lr_adjust = {epoch: learning_rate * (0.5 ** ((epoch - 1) // 1))}
        elif lradj == 'type2':
            lr_adjust = {
                2: 5e-5, 4: 1e-5, 6: 5e-6, 8: 1e-6,
                10: 5e-7, 15: 1e-7, 20: 5e-8
            }
        elif lradj == "cosine":
            lr_adjust = {epoch: learning_rate / 2 * (1 + math.cos(epoch / train_epochs * math.pi))}
        if epoch in lr_adjust.keys():
            lr = lr_adjust[epoch]
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
            print('Updating learning rate to {}'.format(lr))

    # 设置保存模型路径
    if not os.path.exists(model_path):
        os.makedirs(model_path)

    # 定义模型和损失函数
    generator = generator_name(**generator_args).to(device)
    discriminator = discriminator_name(**discriminator_args).to(device)

    optimizer_G = optim.Adam(generator.parameters(), lr=learning_rate, betas=(0.5, 0.999))
    optimizer_D = optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(0.5, 0.999))
    criterion = loss

    # 损失函数值
    G_losses, D_losses = [], []

    for epoch in tqdm(range(n_epochs)):
        total_G_loss = 0
        total_D_loss = 0
        for i, (imgs, _) in enumerate(data_loader):
            # 将输入数据移至device
            imgs = imgs.to(device)

            # 对真实图像添加噪声
            noisy_real_imgs = add_noise(imgs, noise_level=noise_level)

            # 创建真实标签和假标签
            # real_labels = torch.ones(imgs.size(0), 1).to(device)
            # fake_labels = torch.zeros(imgs.size(0), 1).to(device)
            # 使用标签平滑
            real_labels = torch.full((imgs.size(0), 1), 0.9).to(device)  # 真实标签平滑到 0.9
            fake_labels = torch.full((imgs.size(0), 1), 0.1).to(device)  # 假标签平滑到 0.1

            # 生成假图像
            z = torch.randn(imgs.size(0), latent_dim).to(device)
            fake_imgs = generator(z)
            noisy_fake_imgs = add_noise(fake_imgs, noise_level=noise_level)

            # 判别器损失计算
            real_loss = criterion(discriminator(noisy_real_imgs), real_labels)
            fake_loss = criterion(discriminator(noisy_fake_imgs.detach()), fake_labels)
            loss_D = real_loss + fake_loss

            optimizer_D.zero_grad()
            loss_D.backward()
            torch.nn.utils.clip_grad_norm_(discriminator.parameters(), clip_value)
            optimizer_D.step()

            # 训练生成器
            loss_G = criterion(discriminator(noisy_fake_imgs), real_labels)

            optimizer_G.zero_grad()
            loss_G.backward()
            torch.nn.utils.clip_grad_norm_(generator.parameters(), clip_value)
            optimizer_G.step()

            # 记录损失
            total_G_loss += loss_G.item()
            total_D_loss += loss_D.item()

        # 计算每个epoch的损失平均
        avg_G_loss = total_G_loss / len(data_loader)
        avg_D_loss = total_D_loss / len(data_loader)

        # 记录所有epoch的loss
        G_losses.append(avg_G_loss)
        D_losses.append(avg_D_loss)

        # 打印训练过程
        print(
            f'Epoch [{epoch}/{n_epochs}], Generator Loss: {avg_G_loss:.4f}, Discriminator Loss: {avg_D_loss:.4f}')

        # 每个 epoch 保存生成的图片
        fake_imgs = fake_imgs.view(fake_imgs.size(0), channels, latent_dim, latent_dim)
        vutils.save_image(fake_imgs[:25], f"{image_path}/output_epoch_{epoch+1}.png", nrow=5, normalize=True)

        # 调整学习率
        adjust_learning_rate(optimizer_G, epoch + 1, lradj, learning_rate, n_epochs)
        adjust_learning_rate(optimizer_D, epoch + 1, lradj, learning_rate, n_epochs)

    # 绘制损失函数图
    def plot_loss(G_losses, D_losses):
        plt.figure(figsize=(10, 5))
        plt.style.use('seaborn-v0_8-paper')
        plt.grid(axis='y', linewidth=0.35)
        plt.plot(G_losses, linestyle='-', color='#11b3b6')
        plt.plot(D_losses, linestyle='-', color='#f14643')
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.title("Training and Validation Progress")
        plt.legend(["Generator", "Discriminator"])
        plt.show()

    if plots:
        plot_loss(G_losses, D_losses)

    return generator


In [ ]:
# 定义生成器网络
class Generator(nn.Module):
    def __init__(self, latent_dim, channels):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            # 1. 全连接层：将潜在向量 (latent_dim) 投影并展平为形状 (128, 8, 8)
            nn.Linear(latent_dim, 128 * 8 * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Unflatten(1, (128, 8, 8)),  # 转换为形状 (batch, 128, 8, 8)

            # 2. 转置卷积层：第一次上采样，(128, 8, 8) -> (128, 16, 16)
            nn.ConvTranspose2d(128, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            # 3. 转置卷积层：第二次上采样，(128, 16, 16) -> (64, 32, 32)
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),

            # 4. 卷积层：输出通道调整为图像通道数 (64, 32, 32) -> (3, 32, 32)
            nn.Conv2d(64, channels, kernel_size=3, stride=1, padding=1),
            nn.Tanh()  # 将输出范围限制在 [-1, 1]
        )

    def forward(self, z):
        return self.model(z)

# 定义判别器网络
class Discriminator(nn.Module):
    def __init__(self, channels):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            # 1. 卷积层：初始通道从图像通道数增加到 128，(3, 32, 32) -> (128, 32, 32)
            nn.Conv2d(channels, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            # 2. 卷积层：下采样，(128, 32, 32) -> (128, 16, 16)
            nn.Conv2d(128, 128, 4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            # 3. 卷积层：下采样，(128, 16, 16) -> (128, 8, 8)
            nn.Conv2d(128, 128, 4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            # 4. 卷积层：下采样，(128, 8, 8) -> (128, 4, 4)
            nn.Conv2d(128, 128, 4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            # 5. 展平层：将 (128, 4, 4) 展平为向量 (128 * 4 * 4)
            nn.Flatten(),
            nn.Dropout(0.4),  # 添加 Dropout 防止过拟合

            # 6. 全连接层：映射到单个输出值 (即概率)
            nn.Linear(128 * 4 * 4, 1),
            nn.Sigmoid()  # 使用 Sigmoid 输出范围在 [0, 1]
        )

    def forward(self, img):
        return self.model(img)